<a href="https://colab.research.google.com/github/RocketRikky/Secure_and_Private_AI/blob/master/Pysyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch as th

In [141]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [143]:
y

tensor([ 2,  4,  6,  8, 10])

In [144]:
!pip install syft

In [0]:
import syft as sy

In [146]:
hook = sy.TorchHook(th)

W0902 07:04:00.824199 139747537282944 hook.py:102] Torch was already hooked... skipping hooking process


In [147]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Basic Remote Execution in Pysyft

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [149]:
bob._objects

{17775482007: tensor([[-0.0454],
         [ 0.0295]], grad_fn=<AddmmBackward>), 20368643338: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 36035447383: tensor([1, 2, 3, 4, 5]), 50613073430: tensor([[0.],
         [0.]], requires_grad=True)}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [152]:
bob._objects

{17775482007: tensor([[-0.0454],
         [ 0.0295]], grad_fn=<AddmmBackward>), 20368643338: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 36035447383: tensor([1, 2, 3, 4, 5]), 50613073430: tensor([[0.],
         [0.]], requires_grad=True), 72430898549: tensor([1, 2, 3, 4, 5])}

In [153]:
x.location == bob

True

In [154]:
x.id_at_location

72430898549

In [155]:
x.id

82219813548

In [156]:
x.owner

<VirtualWorker id:me #objects:0>

In [157]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [158]:
x

(Wrapper)>[PointerTensor | me:82219813548 -> bob:72430898549]

In [159]:
x=x.get()
x

tensor([1, 2, 3, 4, 5])

In [160]:
bob._objects

{17775482007: tensor([[-0.0454],
         [ 0.0295]], grad_fn=<AddmmBackward>), 20368643338: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 36035447383: tensor([1, 2, 3, 4, 5]), 50613073430: tensor([[0.],
         [0.]], requires_grad=True)}

Playing with Remote Tensors

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [162]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(bob, alice)

In [164]:
bob._objects

{17775482007: tensor([[-0.0454],
         [ 0.0295]], grad_fn=<AddmmBackward>), 20368643338: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 36035447383: tensor([1, 2, 3, 4, 5]), 50613073430: tensor([[0.],
         [0.]], requires_grad=True), 58111196323: tensor([1, 2, 3, 4, 5])}

In [165]:
alice._objects

{13421192456: tensor([1, 2, 3, 4, 5]), 25385839670: tensor([[1.],
         [1.]], requires_grad=True), 45145151505: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 50624185629: tensor([1, 2, 3, 4, 5]), 53896541000: tensor([[0.8567],
         [1.1119]], grad_fn=<AddmmBackward>)}

In [166]:
x.owner

<VirtualWorker id:me #objects:0>

In [167]:
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:25937396945 -> bob:58111196323]
	-> [PointerTensor | me:18594702426 -> alice:50624185629]

In [168]:
x = x.get()
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [169]:
bob._objects

{17775482007: tensor([[-0.0454],
         [ 0.0295]], grad_fn=<AddmmBackward>), 20368643338: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 36035447383: tensor([1, 2, 3, 4, 5]), 50613073430: tensor([[0.],
         [0.]], requires_grad=True)}

In [170]:
alice._objects

{13421192456: tensor([1, 2, 3, 4, 5]), 25385839670: tensor([[1.],
         [1.]], requires_grad=True), 45145151505: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 53896541000: tensor([[0.8567],
         [1.1119]], grad_fn=<AddmmBackward>)}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [172]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [174]:
x

(Wrapper)>[PointerTensor | me:45547117712 -> bob:78930870444]

In [175]:
y

(Wrapper)>[PointerTensor | me:90988216085 -> bob:20038533352]

In [0]:
z = x + y

In [177]:
z

(Wrapper)>[PointerTensor | me:18250062571 -> bob:32151894157]

In [178]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [179]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:32739321942 -> bob:76524063325]

In [180]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [183]:
z.backward()

(Wrapper)>[PointerTensor | me:49745073888 -> bob:21348972527]

In [184]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [185]:
x.grad

tensor([1., 1., 1., 1., 1.])

Learn a Simple Linear Model

In [0]:
input1 = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [187]:
input1

(Wrapper)>[PointerTensor | me:99546600011 -> bob:92565828963]

In [188]:
target

(Wrapper)>[PointerTensor | me:66626630843 -> bob:58806848401]

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [190]:
for i in range(10):  
  
  pred = input1.mm(weights)

  loss = ((pred - target)**2).sum()
  
  loss.backward()
  
  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


Garbage Collection and Common Errors

In [0]:
bob = bob.clear_objects()

In [192]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [194]:
bob._objects

{54887507717: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [196]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [198]:
x.child.garbage_collect_data

True

In [199]:
bob._objects

{13315547982: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [201]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [203]:
x

(Wrapper)>[PointerTensor | me:87944264095 -> bob:61871718047]

In [204]:
bob._objects

{61871718047: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [206]:
bob._objects

{61871718047: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [208]:
bob._objects

{61871718047: tensor([1, 2, 3, 4, 5])}

In [209]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
  x = th.tensor([1,2,3,4,5]).send(bob)

Toy Federated Learning

In [0]:
from torch import nn, optim

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [0]:
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [215]:
def train(iterations=20):
  for iter in range(iterations):
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.data)
    
train()

tensor(3.1272)
tensor(1.1772)
tensor(0.7390)
tensor(0.5083)
tensor(0.3559)
tensor(0.2514)
tensor(0.1790)
tensor(0.1284)
tensor(0.0927)
tensor(0.0674)
tensor(0.0493)
tensor(0.0363)
tensor(0.0268)
tensor(0.0199)
tensor(0.0149)
tensor(0.0111)
tensor(0.0084)
tensor(0.0063)
tensor(0.0048)
tensor(0.0036)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [220]:
_data, _target = datasets[0]
_data

(Wrapper)>[PointerTensor | me:60207879816 -> bob:15111458010]

In [0]:
def train(iterations=20):
  
  model = nn.Linear(2,1)
  opt = optim.SGD(params=model.parameters(), lr=0.1)
  
  for iter in range(iterations):

    for _data, _target in datasets:
      model = model.send(_data.location)

      opt.zero_grad()

      pred = model(_data)

      loss = ((pred - _target)**2).sum()

      loss.backward()

      opt.step()

      model = model.get()
  
      print(loss.get())

In [222]:
train()

tensor(4.5501, requires_grad=True)
tensor(0.0388, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0027, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(9.9063e-05, requires_grad=True)
tensor(8.2452e-05, requires_grad=True)
tensor(6.8626e-05, requires_grad=True)
tensor(5.7118e-05, requires_grad=True)
tensor(4.7539e-05, requires_grad=True)
tensor(3.9568e-05, requires_grad=True)
tensor(3.2933e-05, requires_gra

In [0]:
#list(model.parameters())

Advanced Remote Execution Tools

In [224]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [225]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [226]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:94876423252 -> bob:19540000931]

In [0]:
x = x.send(alice)

In [228]:
bob._objects

{19540000931: tensor([1, 2, 3, 4, 5])}

In [229]:
alice._objects

{94876423252: (Wrapper)>[PointerTensor | alice:94876423252 -> bob:19540000931]}

In [230]:
x

(Wrapper)>[PointerTensor | me:96209235452 -> alice:94876423252]

In [0]:
y = x + x

In [232]:
y

(Wrapper)>[PointerTensor | me:44133723287 -> alice:52010814793]

In [233]:
bob._objects

{19540000931: tensor([1, 2, 3, 4, 5]),
 52664478235: tensor([ 2,  4,  6,  8, 10])}

In [234]:
alice._objects

{52010814793: (Wrapper)>[PointerTensor | alice:52010814793 -> bob:52664478235],
 94876423252: (Wrapper)>[PointerTensor | alice:94876423252 -> bob:19540000931]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [237]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:5723010485 -> bob:39321470421]

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [239]:
bob._objects

{88363085947: tensor([1, 2, 3, 4, 5]), 90938883184: tensor([1, 2, 3, 4, 5])}

In [240]:
alice._objects

{13290698462: (Wrapper)>[PointerTensor | alice:13290698462 -> bob:90938883184],
 33255572958: (Wrapper)>[PointerTensor | alice:33255572958 -> bob:88363085947]}

In [241]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:33255572958 -> bob:88363085947]

In [242]:
alice._objects

{13290698462: (Wrapper)>[PointerTensor | alice:13290698462 -> bob:90938883184]}

In [243]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

Pointer Chain Operations

In [244]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [246]:
bob._objects

{8663948943: tensor([1, 2, 3, 4, 5])}

In [247]:
alice._objects

{}

In [248]:
x.move(alice)

(Wrapper)>[PointerTensor | me:88194958461 -> alice:86088579779]

In [249]:
bob._objects

{}

In [250]:
alice._objects

{86088579779: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [252]:
bob._objects

{75202658226: tensor([1, 2, 3, 4, 5])}

In [253]:
alice._objects

{43523288081: (Wrapper)>[PointerTensor | alice:43523288081 -> bob:75202658226],
 86088579779: tensor([1, 2, 3, 4, 5])}

In [254]:
x.remote_get()

(Wrapper)>[PointerTensor | me:32885695534 -> alice:43523288081]

In [255]:
bob._objects

{}

In [256]:
alice._objects

{43523288081: tensor([1, 2, 3, 4, 5]), 86088579779: tensor([1, 2, 3, 4, 5])}

In [257]:
x.move(bob)

(Wrapper)>[PointerTensor | me:6156871313 -> bob:32885695534]

In [258]:
x

(Wrapper)>[PointerTensor | me:6156871313 -> bob:32885695534]

In [259]:
bob._objects

{32885695534: tensor([1, 2, 3, 4, 5])}

SECURING FEDERATED LEARNING

Project: Federated Learning with a Trusted Aggregator

In [260]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0902 07:04:03.454004 139747537282944 hook.py:102] Torch was already hooked... skipping hooking process


In [0]:
#create a couple of workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [262]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0902 07:04:03.474580 139747537282944 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0902 07:04:03.476959 139747537282944 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0902 07:04:03.478661 139747537282944 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0902 07:04:03.482336 139747537282944 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0902 07:04:03.483834 139747537282944 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0902 07:04:03.485683 139747537282944 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:4>

In [0]:
# A toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# Get pointers to training data on each worker by 
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)


In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Initialize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)

In [0]:
alices_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [270]:
for i in range(10):
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(bobs_data)
  bobs_loss = ((bobs_pred - bobs_target)**2).sum()
  bobs_loss.backward()

  bobs_opt.step()
  bobs_loss = bobs_loss.get().data
  print(bobs_loss)
  
  alices_opt.zero_grad()
  alices_pred = alices_model(alices_data)
  alices_loss = ((alices_pred - alices_target)**2).sum()
  alices_loss.backward()

  alices_opt.step()
  alices_loss = alices_loss.get().data
  print(alices_loss)

tensor(0.4297)
tensor(3.9132)
tensor(0.1109)
tensor(0.0320)
tensor(0.0365)
tensor(0.0018)
tensor(0.0180)
tensor(0.0013)
tensor(0.0124)
tensor(0.0011)
tensor(0.0099)
tensor(0.0009)
tensor(0.0083)
tensor(0.0008)
tensor(0.0070)
tensor(0.0006)
tensor(0.0060)
tensor(0.0005)
tensor(0.0051)
tensor(0.0004)


In [0]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [272]:
secure_worker._objects

{8060375486: Parameter containing:
 tensor([[ 0.6295, -0.0267]], requires_grad=True),
 17003801662: Parameter containing:
 tensor([0.3854], requires_grad=True),
 17403923274: Parameter containing:
 tensor([0.7186], requires_grad=True),
 22072383185: Parameter containing:
 tensor([0.0559], requires_grad=True),
 64755854191: Parameter containing:
 tensor([[-0.1444,  0.0690]], requires_grad=True),
 68171981891: Parameter containing:
 tensor([[0.1506, 0.2328]], requires_grad=True),
 86741182830: Parameter containing:
 tensor([-0.0422], requires_grad=True),
 98758672733: Parameter containing:
 tensor([[ 0.0323, -0.0909]], requires_grad=True)}

In [0]:
model.weight.data = ((alices_model.weight.data + bobs_model.weight.data) / 2).get()

In [0]:
model.bias.data = ((alices_model.bias.data + bobs_model.bias.data) / 2).get()

In [277]:
for round_iter in range(10):
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

  for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
  
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
  
  alices_model.move(secure_worker)
  bobs_model.move(secure_worker)

  model.weight.data = ((alices_model.weight.data + bobs_model.weight.data) / 2).get()
  model.bias.data = ((alices_model.bias.data + bobs_model.bias.data) / 2).get()

  secure_worker.clear_objects()

  print("Bob:" + str(bobs_loss) + "Alice:" + str(alices_loss))

Bob:tensor(0.0025)Alice:tensor(0.0003)
Bob:tensor(0.0019)Alice:tensor(0.0004)
Bob:tensor(0.0014)Alice:tensor(0.0003)
Bob:tensor(0.0011)Alice:tensor(0.0002)
Bob:tensor(0.0008)Alice:tensor(0.0001)
Bob:tensor(0.0006)Alice:tensor(0.0001)
Bob:tensor(0.0005)Alice:tensor(8.0026e-05)
Bob:tensor(0.0004)Alice:tensor(6.0062e-05)
Bob:tensor(0.0003)Alice:tensor(4.5135e-05)
Bob:tensor(0.0002)Alice:tensor(3.3938e-05)


Build Methods for Encrypt, Decrypt and Add

In [0]:
import random

In [0]:
x = 5

In [0]:
Q = 8643286428019047225236

In [0]:
def encrypt(x, n_shares):
  n_shares = 3
  shares = list()

  for i in range(n_shares-1):
       shares.append(random.randint(0,Q))
    
  final_share = Q - (sum(shares) % Q) + x    
  shares.append(final_share)
  
  
  return tuple(shares)  

In [293]:
encrypt(5, n_shares=10)

(4368365095416890693455, 3008812267533009550888, 1266109065069146980898)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [296]:
decrypt(encrypt(5, 3))

5

In [0]:
def add(a, b):
  
  c = list()
  for i in range(len(a)):
    
    c.append((a[i] + b[i]) % Q)
    
  return tuple(c)

In [300]:
decrypt(add(encrypt(5,3), encrypt(10,3)))

15